<p align="center">
  <img src="GitDocuBannerV3.png" alt="StealTheFiles Banner" width="50%">
</p>

# Dokumentation

Wintersemester 2025/26 – HSBI Campus Minden Programmieren 1 - Testat –
Autoren: Tom Coombs, Leonardo Rosario Parrino

### Aufteilung der Arbeit

-   Spielidee & Kreatives: Leonardo Parrino
-   Technische Konzepte & Struktur-Vorschläge: Tom Coombs
-   Technische Grundstruktur (Gui, Input Handling, Game-Loop etc..): Tom
    Coombs
-   Dokumentation: Leonardo Parrino (kleinere Teil auch von Tom Coombs)

------------------------------------------------------------------------

## Inhaltsverzeichnis

-   [1. Konzept & Projektidee](#1-konzept--projekteidee)
-   [2. Entwicklungsumgebung & genutzte
    Dependencies](#2-entwicklungsumgebung--genutzte-dependencies)
    -   [2.1 Entwicklungsumgebung & Java
        Versionen](#21-entwicklungsumgebung--java-versionen)
    -   [2.2 Maven Dependencies](#22-maven-dependencies)
-   [3. LLMs](#3-llms)
-   [4. Softwarearchitektur &
    Design-Patterns](#4-softwarearchitektur--design-patterns)
    -   [4.1. Custom Event-Bus
        (Reflection-Based)](#41-custom-event-bus-reflection-based)
    -   [4.2. Logging-Strategie](#42-logging-strategie)
    -   [4.3. Game-Loop & Threading](#43-game-loop--threading)
    -   [4.4. Entity-System &
        Serialisierung](#44-entity-system--serialisierung)
-   [5. Datenhaltung, Audio &
    Networking](#5-datenhaltung-audio--networking)
    -   [5.1. Audio-Engine (Ducking &
        Caching)](#51-audio-engine-ducking--caching)
    -   [5.2. Multithreaded Networking](#52-multithreaded-networking)
    -   [5.3. Internationalisierung
        (I18n)](#53-internationalisierung-i18n)
-   [6. Build & Deployment
    (Verteilung)](#6-build--deployment-verteilung)
    -   [6.1. Custom Runtime Image
        (JLink)](#61-custom-runtime-image-jlink)
    -   [6.2. Native Installer
        (JPackage)](#62-native-installer-jpackage)
-   [7. Herausforderungen & Technische
    Lösungen](#7-herausforderungen--technische-lösungen)
-   [8. Game Assets & Design](#8-game-assets--design)
-   [9. Dokumentation für das GUI &
    LevelDesign](#9-dokumentation-aufbau-von-gui-und-level-architektur)
    -   [9.1. GUI-Rendering mit JavaFX](#91-gui-rendering-mit-javafx)
        -   [9.1.1. Kurze Erklärung zu
            JavaFX](#911-kurze-erklärung-zu-javafx)
        -   [9.1.2. Das vereinfachte erstellen der einzelnen GUIs durch
            die `UIUtils`
            Klasse](#912-das-vereinfachte-erstellen-der-einzelnen-guis-durch-die-uiutils-klasse)
    -   [9.2. Das `GuiScreen`-Konzept](#92-das-guiscreen-konzept)
    -   [9.3. Level-Implementierung über den
        GameScreen](#93-level-implementierung-über-den-gamescreen)

------------------------------------------------------------------------

## 1. Konzept & Projekteidee

-   Allgemein soll es einen einfachen 2D-Platformer abbilden.
-   Es gibt einen Spieler, welcher versucht bis zum Level-Ziel zu kommen
    und dabei so viele Ordner (mit Klausuren) wie möglich einsammelt.
    -   Dieser Ordner befinden sich über das ganze Level verteilt.
-   Die Steuerung soll sich mit `A`, `D` und `SPACE` einfach halten und
    ein intuitives Spielerlebnis bieten.
-   Geplant war auch die Umsetzung eines Mehrspielermodus, welcher
    ermöglichen soll, in einer großen gemeinsamen Map zu spielen.
-   Projektstruktur:
    -   Da ein Multiplayer geplant war, haben wir das Projekt in
        mehreren Modulen aufgebaut.
        -   Das `core` Modul beinhaltet alle Funktionen und Klaseen,
            welche vom Client sowie vom Server benötigt werden.
        -   Das `client` Modul ist das, was man als das Spiel verstehen
            würde. Der Client beinhaltet alle JavaFX Komponenten,
            verfügt jedoch nicht über eine Datenbankverbindung um Stats
            direkt selber zu speichern.
        -   Das `server` Modul ist sozusagen der GameServer. Dieser
            handelt das gesamte Multiplayer verhalten sowie alle
            Datenbankanfragen für z.B. Stats.
        -   Die Grundidee für diese Aufteilung in verschiedene Module
            ist, dass man zum einen für den Client und Server die selben
            Klassen für z.B. `Entities` oder `Locations` nutzen kann.
            Dazu bietet ein getrenntes Servermodul die Möglichkeit, das
            der GameServer unabhänig von einem Client allein laufen
            kann, und somit nicht jeder Client über sein Heimnetzwerk
            den gesamten Netzwerktraffic laufen lassen muss.

## 2. Entwicklungsumgebung & genutzte Dependencies

### 2.1 Entwicklungsumgebung & Java Versionen

-   **IDE:** IntelliJ IDEA (Community Edition)
-   **Java Version:** Amazon Corretto SDK 19.0.2
-   **JavaFX Version:** SDK-17.0.6

### 2.2 Maven Dependencies

| Artefakt            | Link                                                                            | Beschreibung                                                                                       |
|:--------------------|:--------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------|
| **Log4j**           | [mvnrepository](https://mvnrepository.com/artifact/log4j/log4j)                 | Logging-Framework für Fehler- und Statusmeldungen.                                                 |
| **Lombok**          | [mvnrepository](https://mvnrepository.com/artifact/org.projectlombok/lombok)    | Reduziert Boilerplate-Code (Getter, Setter, Konstruktoren) durch Annotationen.                     |
| **MySQL Connector** | [mvnrepository](https://mvnrepository.com/artifact/mysql/mysql-connector-java)  | JDBC-Treiber für die Verbindung zur MySQL-Datenbank.                                               |
| **HikariCP**        | [mvnrepository](https://mvnrepository.com/artifact/com.zaxxer/HikariCP)         | Performanter JDBC Connection Pool für effiziente Datenbankverbindungen.                            |
| **Gson**            | [mvnrepository](https://mvnrepository.com/artifact/com.google.code.gson/gson)   | Bibliothek zur Serialisierung und Deserialisierung von JSON-Daten.                                 |
| **JavaFX Controls** | [mvnrepository](https://mvnrepository.com/artifact/org.openjfx/javafx-controls) | **Das Hauptmodul für die GUI-Elemente** (Buttons, Labels, Checkboxen, etc.).                       |
| **JavaFX FXML**     | [mvnrepository](https://mvnrepository.com/artifact/org.openjfx/javafx-fxml)     | Ermöglicht das **Deklarieren der Benutzeroberfläche** in XML-Dateien, getrennt von der Java-Logik. |
| **JavaFX Graphics** | [mvnrepository](https://mvnrepository.com/artifact/org.openjfx/javafx-graphics) | Stellt die Kern-API für das Rendern von Grafiken bereit (wird von Controls/FXML benötigt).         |
| **JavaFX Media**    | [mvnrepository](https://mvnrepository.com/artifact/org.openjfx/javafx-media)    | Modul zur Integration von Audio und Video (z.B. Player-Funktionalität).                            |

**Erklärung warum sich für bestimmte Dependencies entschieden wurde:**

-   Lombok: Dieses Tool nutze ich bereits seit vielen Jahren und dadurch
    weiß ich gut, wie hilfreich es sein kann, um einiges ein Code zu
    sparen. Vor allem in Klassen mit vielen Attributen ist Lombok eines
    der besten Tools um eine gute und organisierte Codestruktur zu
    behalten.
-   Log4j: Das Tool ist nur für das Loggen vorhanden. Auch wenn unser
    Projekt nicht allzu viele Stellen hat, wo etwas geloggt wird, ist es
    trotzdem hilfreich, um in späteren Szenarien bereits eine gute
    Codestruktur zu haben, bei welcher die Implementation eines
    Loggingsystem super einfach ist.
-   HikariCP & MySQL: Dieses beiden Dependencies sind im Server-Modul
    vorhanden und dienen schlicht dazu eine Datenbankverbindung
    herzustellen. HikariCP ist dazu der aus meiner Erfahrung mit Abstand
    performanteste JDBC-Verbindungspool und ermöglicht eine stabile
    Datenbankverbindung mit sehr geringer Latenz.
-   Gson: Gson wird einfach nur genutzt um die JsonConfig einfach zu
    Verwalten und um aus einer JSON-Datei eine Javaklasse zu generieren.

## 3. LLMs

Allgemein wurden in diesem Projekt so gut wie keine LLMs zur Generierung
des Quelltextes verwendet. Der Hintergrund ist vor allem, dass ich (Tom)
bereits fast eine Dekade an Programmiererfahrung und viele Jahre in
IT-Unternehmen als Fullstack-Developer habe und somit bereits einiges an
Kennwissen besitze. <br> Trotzdem wurden für Teillösungen LLMs genutzt,
da es zum einen Aufgabe war und zum anderen LLMs wie Copilot, Tabnine
oder Codex genutzt. <br> Dabei konnte auch festgestellt werden, dass
LLMs für einige Probleme sehr hilfreich sind. Vor allem bei Probleme
welche normaler weiße etwas komplexer sind, wie z.B. die Implementierung
der Kamera, konnten die LLMs Teile unserer Probleme bei der
Implementation einer solchen beheben.

**Verwendetes Modell: Gemini 2.0 Flash Thinking**

<details>
<summary><strong>Hier klicken, um die verwendeten Prompts anzuzeigen</strong></summary>

<br>

**Prompt 1: Der Überblick (Design-Fokus)**

> “Ich entwickle einen 2D-Platformer in JavaFX und suche nach einer
> besseren Kamera-Lösung als der starren Spieler-Fixierung. Kannst du
> mir eine Übersicht der gängigen Systeme geben?”

**Prompt 2: Die Technik (Mathematik & Code-Logik)**

> “Meine Kamera wirkt ruckelig und springt immer etwas nach vorne. Ich
> möchte ein flüssiges Verhalten wie bei modernen Platformern wie z.B.
> Hollow Knight erreichen. Kannst du mir die Mathematik hinter ‘Position
> Smoothing’ erklären und wie ich das sauber einbaue? Wichtig ist mir
> auch, dass die Kamera nicht über den Levelrand hinausfährt. Ich möchte
> die Logik verstehen, um sie selbst zu implementieren.”

</details>

Wir hatten zu einem Zeitpunkt während des Entwicklungsprozesses
versucht, dass einige aus unserem Team KI nutzen, jedoch mussten wir
dort schnell feststellen, dass egal welche KI wir benutzt haben, keine
auch nur im Ansatz gut mit der Projektstruktur kalr gekommen ist. Ein
dort sehr häufig zu beobachtendes Verhalten war, dass die LLMs den
großteil des generierten Codes alles in eine große Dateie gesteckt
haben, welches die Organisation und Verständlichkeit des Codes sehr
stark reduziert. Am Ende sind wir bei dem Entschluss geblieben, dass
LLMs genutzt werden können, um kleine Teilprobleme zu lösen, allerdings
bei komplexeren Aufgaben gänzlich versagen. Das beste wofür wir die LLMs
nutzen konnten war für Vorschläge oder Erklärungen, jedoch war selbst
dort eine schnelle suche auf StackOverflow oder anderen Foren meist
hilfreicher.

## 4. Softwarearchitektur & Design-Patterns

Das Projekt folgt einer strikten Trennung zwischen Logik, Daten und
Darstellung (MVC-Ansatz). Neben dem Singleton-Pattern und dem
State-Pattern sind folgende komplexe Systeme zentral:

### 4.1. Custom Event-Bus (Reflection-Based)

Anstatt auf Standard-JavaFX-Events beschränkt zu sein, wurde ein
eigenes, hochflexibles Event-System implementiert.

-   **Funktionsweise:** Über die Annotation `@EventHandler` markierte
    Methoden werden zur Laufzeit mittels **Java Reflection** registriert
    und dynamisch aufgerufen.
-   **Loose Coupling:** Das System entkoppelt Komponenten vollständig.
    Beispiel: Das `PlayerDamageEvent` enthält alle Infos über Schaden
    (`DamageCause`, `newHealth`), ohne dass der Auslöser (z.B. ein
    Gegner) die Spieler-Klasse direkt manipulieren muss.
-   **Cancellable Events:** Events können abgebrochen werden
    (`setCancelled`), um Logik wie Schaden im “GodMode” zu unterbinden.

### 4.2. Logging-Strategie

Zur Fehleranalyse wird **Log4j** eingesetzt. Die Konfiguration
(`log4j.properties`) definiert eine duale Strategie:

-   **Console:** Direkte Ausgabe in die IDE für Entwicklung.
-   **File-Rotation:** Es wird parallel in eine `latest.log` und eine
    timestamp-basierte Datei (`logs/log_${startTime}.log`) geschrieben.
    Dies ermöglicht eine nachträgliche Fehleranalyse bei Abstürzen, ohne
    alte Logs zu überschreiben.

### 4.3. Game-Loop & Threading

Der `ScreenManager` trennt die Programmlogik vom
JavaFX-Rendering-Thread.

-   **Dedizierter Thread:** Ein eigener Thread (`gameLoop`) berechnet
    Physik und Spielzustände.
-   **Synchronisation:** Grafische Updates werden über
    `Platform.runLater()` sicher an die UI übergeben.

### 4.4. Entity-System & Serialisierung

Die Spielobjekte sind netzwerkfähig konzipiert. Jede `Entity` besitzt
eine `UUID` und ist `Serializable`. Dadurch können komplette
Spielerobjekte direkt über den TCP-Stream gesendet werden, was die
Multiplayer-Synchronisation massiv vereinfacht.

## 5. Datenhaltung, Audio & Networking

### 5.1. Audio-Engine (Ducking & Caching)

Der `SoundManager` fungiert als professionelle Audio-Engine.

-   **Caching:** Audiodateien werden beim ersten Zugriff in einer
    `HashMap` gecacht (`mediaCache`), um Nachladeruckler zu vermeiden.
-   **Ducking:** Wenn wichtige Sounds (z.B. Achievements oder Buffs)
    spielen, wird die Hintergrundmusik temporär und thread-safe
    (`synchronized`) leiser geregelt, damit das akustische Feedback klar
    hörbar bleibt.

### 5.2. Multithreaded Networking

Der Netzwerk-Client nutzt einen `ExecutorService` (`CachedThreadPool`),
um Blockaden im Hauptthread zu vermeiden.

-   **ReceiverTask:** Wartet blockierend auf eingehende Pakete
    (`dis.read`) und verarbeitet diese im Hintergrund.
-   **SenderTask:** Sendet asynchron in festen Intervallen (alle 50ms)
    Positionsupdates. Dies entkoppelt die Netzwerklatenz vollständig von
    der Framerate des Spiels.

### 5.3. Internationalisierung (I18n)

Texte sind nicht hard-coded. Der `MessageHandler` lädt Übersetzungen
(DE, EN, RU) aus einer JSON-Datei und unterstützt Platzhalter (z.B.
`{0}`), um Nachrichten dynamisch zur Laufzeit zu generieren.

## 6. Build & Deployment (Verteilung)

Ein technisches Highlight ist der Build-Prozess, der das Spiel
unabhängig von einer vorinstallierten Java-Version macht.

### 6.1. Custom Runtime Image (JLink)

Mittels `jlink` wird eine maßgeschneiderte Java-Runtime erstellt, die
nur die wirklich benötigten Module (`javafx.controls`, `javafx.media`,
etc.) enthält. Dies reduziert die Größe der Anwendung drastisch
(`--strip-debug`, `--compress=2`).

### 6.2. Native Installer (JPackage)

Das Skript `start_app_bundler.bat` nutzt das JDK-Tool `jpackage`, um aus
der JAR-Datei und der Custom-Runtime eine native Windows-Anwendung
(`.exe` / Installer) zu generieren.

-   **Vorteil:** Der Endnutzer muss kein Java installieren; alles
    Notwendige wird mitgeliefert (“Self-Contained Application”).
-   **Obfuscation:** Das Build-Skript verweist auf eine
    `client_obfuscated.jar`, was zeigt, dass der Code vor Decompilierung
    geschützt wurde.

## 7. Herausforderungen & Technische Lösungen

Während der Entwicklung stießen wir auf technische Herausforderungen,
die durch spezifische Algorithmen und mathematische Ansätze gelöst
wurden:

-   **Problem: Performance bei großen Leveln (Rendering)**

    -   *Lösung: View Frustum Culling.* In der Klasse `Platform` wird
        geprüft, ob eine Kachel (Tile) im sichtbaren Kamerabereich
        liegt. Ist sie außerhalb, wird sie unsichtbar geschaltet oder
        gar nicht erst berechnet. Das hält die FPS auch in riesigen
        Leveln stabil.

-   **Problem: Frame-Rate-Unabhängige Animationen**

    -   *Lösung: Delta-Time Accumulator.* Damit Animationen (z.B. Lava
        oder drehende USB-Sticks) auf allen PCs gleich schnell laufen,
        wird `System.nanoTime()` genutzt. Die vergangene Zeit wird
        akkumuliert und Frames werden erst gewechselt, wenn ein
        definierter Schwellenwert überschritten ist.

-   **Problem: Statische Bewegungsmuster**

    -   *Lösung: Mathematische Bewegungsmodelle.*
        -   **Interpolation:** `FloatingPlatformBlock` nutzt
            Vektormathematik, um sich flüssig zwischen zwei Punkten zu
            bewegen (Lerp).

-   **Problem: Gegner und Kollision**

    -   *Lösung: Clamped Vector Movement.* Der `RobotEnemyBlock`
        berechnet die Richtung zum Spieler, begrenzt die Bewegung aber
        strikt auf einen definierten Bereich (`minX`, `maxX`). Dies
        verhindert, dass Gegner blindlings von Plattformen laufen.

-   **Problem: Audio-Konflikte**

    -   *Lösung: Synchronisation.* Da Musik und Soundeffekte in
        unterschiedlichen Threads laufen können, werden kritische
        Audio-Operationen (wie das Ducking) durch ein Lock-Objekt
        (`duckLock`) synchronisiert. Dies verhindert inkonsistente
        Lautstärkepegel bei gleichzeitigen Events.

-   **Problem: Komplexität des Projektes**

    -   Das Projekt wurde von uns für die Vorgegebene Zeit und
        Aufgabenstellung mit einem viel zu komplexen Ansatzt angeganen.
        Wir wollten anstelle eines einfachen Spiels, ein sehr komplexes
        mit viele Funktionen und Erweiterungen, wie z.B. eine
        Multiplayer Funktion.
    -   *Lösung:* Da wir bereits viele weitere Funktionen wie den
        `FlipperBlock/Item` oder den `Multiplayer` angefangen hatten,
        wollten wir diese Code-Stücke einfach erstmal behalten für den
        Falle, dass man später in höheren Semestern oder privat das
        Projekt weiter entwickeln möchte.

-   **Problem: Arbeit als Gruppe**

    -   Das wohl mitabstand größte Problem war, dass unsere Gruppe
        Erfahrungsniveau auf komplett verschiedenen Leveln ist. Dies
        stellte bei der Entwicklung einige Probleme dar.
        -   Ein gutes Beispiel dafür ist die Projektstruktur. Für alle
            die aus unserer Gruppe die bisher nur sehr wenig Erfahrung
            in diesem Bereich haben, ist ein so großes Projekt allein
            aufgrund der schieren Größe sehr erdrückend. Das Projekt ist
            für Leute ohne Erfahrung viel zu komplex gewesen. Deswegen
            hat es bereits einige Stunden gedauert, den einzelnen Leuten
            aus der Gruppe zu erklären, wie genau das Projekt aufgebaut
            ist und wie man dieses erweitern kann.
        -   Ein weiteres Problem was dadurch entstanden ist, ist, dass
            all die ohne Erfahrung oft vorhanden Code strukturen zur
            einfachen Erweiterung z.B. den `KeyListener` nicht verwendet
            haben, sondern alles in eine Klasse gepackt haben. Dies
            erforderte oft viel Arbeit um alles geordnet und getrennt zu
            halten.
    -   *Lösung:* Die einfachste Lösung, die wir dafür hatten, war fast
        alles einmal neuzuschreiben und dabei nur wenige der Kommentare
        im Code zu übernehmen.

## 8. Game Assets & Design

Die visuelle Gestaltung des Spiels basiert auf einem eigens erstellten
Sprite-Sheet, das alle relevanten Spielobjekte in einer einzigen Textur
bündelt. Dies reduziert “Draw Calls” und verbessert die Performance
erheblich.

<p align="center">
  <img src="GameAssetsStealTheFile.png" alt="Game Assets Übersicht" width="40%">
</p>

Das Asset-Pack umfasst folgende Elemente:

-   **Charaktere:**

    -   **Player-Figur:** Der Protagonist des Spiels.
    -   **Robot Enemy:** Der Gegner wurde inspiriert durch das Design
        auf der [HSBI Informatik-Bachelor
        Seite](https://www.hsbi.de/studiengaenge/informatik-bachelor).

-   **Umgebung & Setting:**

    -   **Server-Schränke:** Um das Setting authentisch zu gestalten,
        wurden zwei Varianten erstellt, deren Texturen auf Fotos des
        [HSBI Labors für Angewandte
        Informatik](https://www.hsbi.de/ium/studium/labore/angewandte-informatik)
        basieren.
    -   **Plattformen:** Grafiken für statische Böden sowie **schwebende
        Plattformen**.
    -   **Gefahren:** Grüne Säure/Lava als Hindernis.
    -   **Level-Elemente:** Ein **Portal** für den Level-Abschluss.

-   **Items & UI:**

    -   **USB-Stick:** Das Sammelobjekt (Ziel des Spiels).
    -   **Power-Ups:** Speedboost und Jumpboost.
    -   **UI:** Eine integrierte Lebensanzeige (Health Bar).

## 9. Dokumentation: Aufbau von GUI und Level-Architektur

In diesem Abschnitt wird der schrittweise Aufbau der **Graphical User
Interface (GUI)** sowie die architektonische Umsetzung der **Levels**
dokumentiert. Wichtige Design- und technische Entscheidungen werden
kommentiert.

------------------------------------------------------------------------

## 9.1. GUI-Rendering mit JavaFX

Die gesamte grafische Oberfläche des Spiels wird mithilfe der
**JavaFX-Bibliothek** gerendert.

### 9.1.1 Kurze Erklärung zu JavaFX

**JavaFX** ist eine Java-Bibliothek, die speziell für die Entwicklung
von Desktop-Anwendungen und Rich Internet Applications (RIA) konzipiert
wurde. Im Vergleich zu älteren Technologien wie Swing bietet JavaFX eine
**modernere, hardwarebeschleunigte Oberfläche** und unterstützt **CSS**
für einfaches Styling.

-   **Technische Notiz:** JavaFX wurde gewählt, da es **leistungsstarkes
    2D-Rendering** und eine klare Trennung von Logik und Darstellung
    (mittels FXML/CSS) ermöglicht, was die Entwicklung der
    Benutzeroberfläche vereinfacht und beschleunigt.

### 9.1.2 Das vereinfachte erstellen der einzelnen GUIs durch die `UIUtils` Klasse

Die UIUtils-Klasse dient dazu, die Erstellung von Benutzeroberflächen zu
vereinfachen und Wiederholungen im Code zu vermeiden (DRY-Prinzip). Sie
kapselt komplexe JavaFX-Abläufe wie die reaktive Zentrierung von
Elementen oder die Implementierung von Hintergrundanimationen in
einfache Methoden wie `UIUtils#drawCenteredButton()`. Dadurch muss diese
Logik nicht in jedem Menü-Screen neugeschrieben werden. Zudem
gewährleistet die Klasse eine konsistente Benutzererfahrung, indem sie
beispielsweise automatisch einen Klick-Sound für alle Buttons hinzufügt.
Dies macht den Code in den eigentlichen Screens sauberer, wartbarer und
fokussierter auf die spezifische Level- oder Menü-Logik. <br> Ein
anschauliches Beispiel dafür wäre die Methode `UIUtils#drawRect(..)`,
mit welcher das Zeichnen eines Rechtecks um einiges vereinfacht wird.

``` java
public static Rectangle drawRect(Pane parent, double x, double y, double width, double height, Color color) {
    Rectangle rect = new Rectangle(x, y, width, height);
    rect.setFill(color);
    parent.getChildren().add(rect);
    return rect;
}
```

Solche Methoden zur Vereinfachung gibt es dazu auch für Textelemente,
Buttons etc.

------------------------------------------------------------------------

## 9.2. Das `GuiScreen`-Konzept

Um die **verschiedenen Zustände** und Ansichten des Fensters (z. B.
Hauptmenü, Einstellungen, eigentliches Spiel) sauber voneinander trennen
und effizient wechseln zu können, wurde das **`GuiScreen`-Konzept**
implementiert.

### Designentscheidung: `GuiScreen`

Die Entscheidung für dieses konzeptionelle Framework ermöglicht eine
klare **Trennung der Zuständigkeiten (Single Responsibility
Principle)**. Jede Ansicht (z. B. das Hauptmenü) wird zu einer
**eigenständigen Klasse** (`MainMenuScreen`), die nur für ihre
spezifische Logik und Darstellung verantwortlich ist.

-   **Struktur:** Jede `GuiScreen`-Instanz besitzt die folgenden
    zentralen Methoden:
    -   `initialize()`: Wird einmalig beim Erstellen des Screens
        ausgeführt, um alle zu rendernden Komponenten erstmalig zu
        erstellen (z. B. Buttons, Textfelder).
    -   `draw()`: Zeichnet die erstellten Komponenten auf den
        Bildschirm.
    -   `update()`: Wird **jeden Frame** ausgeführt, um Logik wie
        Animationen, Eingabeverarbeitung oder Zustandsprüfungen
        darzustellen.

### Implementierung des Screen-Wechsels

Der Wechsel zwischen den `GuiScreen`s wird über einen zentralen
**`ScreenManager`** gesteuert.

``` java
public class ScreenManager {
    @Getter
    private GuiScreen currentScreen;
    // ..

    public void showScreen(GuiScreen screen) {
        // Setze aktuellen Screen
        this.currentScreen = screen;

        /*
         * Wenn screen noch nie geladen wurde -> screen erstmals laden
         */
        if (!this.screenList.contains(screen)) {
            screen.initialize();
        }

        // Screen in Liste hinzufügen und als Root setzen (anzeigen)
        this.screenList.add(screen);
        Platform.runLater(() -> stage.getScene().setRoot(screen.getRoot()));
    }

    //..
}
```

## 9.3. Level-Implementierung über den `GameScreen`

Für das eigentliche Spiel-Gameplay wurde die Architektur so gestaltet,
dass nicht für jedes Level ein neuer `GuiScreen` erstellt wird, sondern
ein **einheitlicher `GameScreen`** als Container dient.

### Designentscheidung: Einheitlicher `GameScreen`

Anstatt viele Level-spezifische Screens zu erstellen (z. B.
`Level1Screen`, `Level2Screen`), fungiert der **`GameScreen`** als
**zentrale Spiel-Umgebung**. Dies vereinfacht das **Laden und Entladen
von Level-Daten** und stellt sicher, dass Elemente wie das **HUD
(Head-Up Display)** oder die **Pausenfunktion** konsistent über alle
Level hinweg funktionieren.

-   **Workflow beim Levelstart:** Der `GameScreen` ist dafür
    verantwortlich, die **Level-Daten** zu laden, die **HUD-Elemente**
    zu instanziieren und die **Spiel-Loop** zu starten.

### Implementierung des Level-Ladens

Der `GameScreen` orchestriert das Laden des aktuellen Levels und der
Benutzeroberfläche (HUD).

``` java
public class GameScreen implements GuiScreen {
    // ...
    @Override
    public void initialize() {
        double width = screenManager.getStage().getWidth();
        double height = screenManager.getStage().getHeight();

        // .. HUD laden ..

        /*
         * Entferne alle Bloecke & Platformen des aktuellen Levels und
         * zeichne das aktuelle Level (dort werden Bloecke & Platformen dann neu geladen)
         */
        Game.getInstance().getCurrentLevel().getBlocks().clear();
        Game.getInstance().getCurrentLevel().getPlatforms().clear();
        Game.getInstance().getCurrentLevel().draw(width, height, root);

        // ...
    }

    @Override
    public void update() {
        // .. Update der Spiellogik (Bewegung, Kollision) ..
        

        // .. Update der HUD (Animationen etc.) ..
    }
    // ...
}
```

------------------------------------------------------------------------

**Name:** Tom Coombs **Matrikelnummer:** 50212815  
**Zum Projekt:** StealTheFiles

<details>
<summary><strong>Reflexion und Fazit anzeigen</strong></summary>

Das Projekt war für mich trotz meiner Erfahrung tatsächlich besonders
interessant. Das liegt vor allem daran, da ich davor eher wenig mit der
Entwicklung eines eigenen Java-Spiels zu tun hatte. Ebenfalls war es
sehr besonders für mich, mit anderen aus dem ersten Semester daran zu
arbeiten.<br> Trotzdem würde ich sagen, dass das Projekt für mich eher
eine Form des Zeitvertreibs war als ein anspruch volles Projekt, was nun
mal auf die Erfahrung zurückzuführen ist. Vor allem am Anfang, wo die
anderen Leute aus meiner Gruppe noch keinerlei Erfahrung hatten, war es
unglaublich schwer dieses Projekt zu entwickeln und verständlich zu
erklären.

Allgemein war es trotzdem eine gute Abwechslung zu meinen sonstigen
Projekten die mehr im Bereich der Backend-Entwicklung und
Serversicherheit liegen sowie TypeScript-Anwendungen liegen.

</details>

------------------------------------------------------------------------

**Name:** Leonardo Rosario Parrino  
**Matrikelnummer:** 50197039  
**Zum Projekt:** StealTheFiles

<details>
<summary><strong>Reflexion und Fazit anzeigen</strong></summary>

Das Projekt **„StealTheFiles“** stellte für mich weit mehr dar als eine
spielerische Aufgabe; es war der entscheidende Schritt, um die
theoretischen Vorlesungsinhalte in ein funktionierendes Softwaresystem
zu überführen. Besonders die hohe Komplexität der verschiedenen
Subsysteme – von der Bewegungslogik über Kollisionsabfragen bis hin zum
Event-Handling und UI – verdeutlichte mir die Notwendigkeit einer
sauberen **Softwarearchitektur**. Ich habe gelernt, Interdependenzen
zwischen Modulen frühzeitig zu erkennen und technisch sauber zu lösen,
was mein Verständnis für das technische „Big Picture“ nachhaltig
geschärft hat.

Auf methodischer Ebene konnte ich meinen Umgang mit
**Versionsverwaltung** professionalisieren. Die Nutzung von GitHub ging
über bloßes Speichern hinaus: Strukturierte Commits und ein sauberer
Workflow waren essenziell für die Nachvollziehbarkeit und die
kollaborative Entwicklung. Auch die **Teamarbeit** war ein
Schlüsselfaktor; wir haben gelernt, Aufgaben effizient zu parzellieren
und uns bei Engpässen gegenseitig zu unterstützen, was den
Entwicklungsprozess deutlich beschleunigte und professionalisierte.

Ein besonderes Augenmerk legte ich auf den reflektierten Umgang mit
modernen Tools wie **LLMs**. Ich nutzte KI als unterstützendes Werkzeug,
erkannte jedoch schnell die Grenzen und die Gefahr einer “blinden
Übernahme”. Dies schärfte mein Verständnis dafür, dass die
Problemlösungskompetenz und die Kontrolle über den Code stets beim
Entwickler bleiben müssen.

Rückblickend war das Projekt nicht nur kognitiv fordernd, sondern auch
eine Quelle großer Motivation. Das „Flow-Erleben“ während der
Programmierung und das erfolgreiche Zusammenfügen der Komponenten haben
mir ein stabiles technisches Fundament verschafft und meine Begeisterung
für die Softwareentwicklung bestätigt.

</details>